In [2]:
from pyscf import gto, scf, md
import re
import pickle
import numpy as np
import os 
import tensorflow

/Users/benni/opt/anaconda3/envs/ml_env/lib/python3.9/site-packages/pyscf/dft/libxc.py:772: UserWarning: Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, the same to the B3LYP functional in Gaussian and ORCA (issue 1480). To restore the VWN5 definition, you can put the setting "B3LYP_WITH_VWN5 = True" in pyscf_conf.py
  warnings.warn('Since PySCF-2.3, B3LYP (and B3P86) are changed to the VWN-RPA variant, '
2023-08-02 11:52:10.493502: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [23]:
h2 = gto.Mole()
h2.atom = [['H', (0.7, 0, 0)], ['H', (-0.7, 0, 0)]]
h2.basis = 'ccpvdz'
h2.unit = 'B'
h2.build() 
h2._atom

[('H', [0.7, 0.0, 0.0]), ('H', [-0.7, 0.0, 0.0])]

In [24]:
mf = scf.RHF(h2)
mf.kernel()

converged SCF energy = -1.12870944897989


-1.1287094489798908

In [25]:
mycas = mf.CASSCF(2, 2)
myscanner = mycas.nuc_grad_method().as_scanner()

# Generate the integrator
# sets the time step to 5 a.u. and will run for 100 steps
# or for 50 a.u.
myintegrator = md.NVE(myscanner,
                            dt=1,
                            steps=1200,
                            energy_output="BOMD.md.energies",
                            trajectory_output="BOMD.md.xyz",
                            verbose=0).run()

# Note that we can also just pass the CASSCF object directly to
# generate the integrator and it will automatically convert it to a scanner
# myintegrator = pyscf.md.NVE(mycas, dt=5, steps=100)

# Close the file streams for the energy and trajectory.
myintegrator.energy_output.close()
myintegrator.trajectory_output.close()


In [26]:
ticks = 30
coords = [[x, y, 0] for x in range(ticks) for y in range(ticks)]

In [29]:
raw_data = open("../../data/BOMD.md.xyz", "r").read() + "2\n"
raw_data = raw_data.split("MD Time")[1:]
R = []
densities = []
corrs = []
data = {}
#for n in range(len(raw_data)):
for n in [0]:
    print(n)
    data_point = raw_data[n]
    data_point = data_point.split("\n")[1:-2]
    data_point = [re.sub(r'\s+', ',', coords).split(",") for coords in data_point]
    data_point = [[atom, (x, y, z)] for [atom, x, y, z] in data_point]
    mol = gto.M()
    mol.atom = data_point
    mol.basis = "ccpvdz"
    mol.build()
    hf = scf.RHF(mol).run(verbose=0)
    rdm1 = hf.make_rdm1()
    rdm2 = hf.make_rdm2()
    ao_vals = np.array(mol.eval_ao("GTOval_sph", coords))
    n_coords = len(coords)
    n_orbitals = ao_vals.shape[-1]
    #ao_vals_pointwise = [[coords[i].tolist(), ao_vals_pointwise[i].tolist()] for i in range(n_coords)]
    density_for_point = []
    corr_for_point = []
    for i in range(n_coords):
        [x, y, _] = coords[i]
        ao_vals_for_point = ao_vals[i]
        ao_vals_for_point = ao_vals_for_point.flatten()
        z = np.einsum("ij,i,j->", rdm1, ao_vals_for_point, ao_vals_for_point)
        density_for_point.append(z)
        for j in range(n_coords):
            [x1, y1, _] = coords[j]
            ao_vals_for_point_1 = ao_vals[j]
            ao_vals_for_point_1 = ao_vals_for_point_1.flatten()
            rho_corr = np.einsum("iijj,i,i,j,j->", rdm2, ao_vals_for_point_1, ao_vals_for_point_1, ao_vals_for_point_1, ao_vals_for_point_1)
            corr_for_point.append(rho_corr)
    R.append([[x, y, z] for [atom, (x, y, z)] in data_point])
    densities.append(density_for_point)
    corrs.append(corr_for_point)
    for key in ["R", "densities", "corrs"]:
        data[key] = eval(key)
    count = str(n)
    filename = f"md_h2_{str(n+1)}.npz"
    with open(f"md_h2/{filename}", "wb") as file:
        pickle.dump(data, file)
    R = []
    densities = []
    corrs = []
    data = {}

0


In [33]:
keys = ["R", "densities", "corrs"]
data_dict = {key: None for key in keys}
filenames = os.listdir("./md_h2")
filenames.sort()
for f in filenames:
    file = np.load(f"./md_h2/{f}", allow_pickle=True)
    f = np.load("./md_h2/md_h2_1.npz", allow_pickle=True)
    for key in keys:
        if data_dict[key] is not None:
            data_dict[key] = np.concatenate((data_dict[key].astype(np.float32), np.array(f[key]).astype(np.float32)), axis=0)
        else:
            data_dict[key] = np.array(f[key])
total_filename = "md_h2_1_61.npz"
if total_filename not in filenames:
    with open(f"./md_h2/{total_filename}", "wb") as file:
        pickle.dump(data_dict, file)

**Create coords**

In [56]:
total_file = np.load(f"./md_h2/{total_filename}", allow_pickle=True)
R = total_file["R"]
R_mean = R.mean(axis=0)
center = R_mean.mean(axis=0)
R_reshaped = R.reshape((61 * 2, 3))
max_abs_value = np.sqrt((R_reshaped * R_reshaped).sum(axis=-1)).max()
ticks = 30
array = np.linspace(-max_abs_value / 2, max_abs_value, ticks)
coords = np.array([[x, y, 0] for x in array for y in array])
total_file["coords"] = coords
pickle.dump(total_file, open(f"./md_h2/{total_filename}", "wb"))

In [58]:
total_file = np.load(f"./md_h2/{total_filename}", allow_pickle=True)

(900, 3)